In [3]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
tf.keras.backend.clear_session()

In [6]:
subject = 'Acre - Consumo de Cimento (t)'
start_index = 0
split_index = 203 #Referente aos 230 anos de input  
train_split = split_index + 1 - 12*3

In [7]:
data = pd.read_csv('2003_mo_model_input_AC.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Unnamed: 0,Acre - IDH Longevidade,Acre - IDH Renda,Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH Educacao,...,Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Acre - Consumo de Cimento (t)
0,2003-1,0.773065,0.651726,8.722906,0.691320,7.285707e+06,413409.255032,9.318417,6.587220e+06,0.643805,...,8.202881e+10,0.724032,-5331.049150,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,3.260
1,2003-2,0.773454,0.651847,8.718028,0.691617,7.294881e+06,413607.843560,9.319990,6.590338e+06,0.644270,...,8.222716e+10,0.690297,-5318.079644,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,2.191
2,2003-3,0.773843,0.651968,8.713149,0.691914,7.304055e+06,413806.432089,9.321563,6.593455e+06,0.644734,...,8.234287e+10,0.669681,-5436.417870,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,2.154
3,2003-4,0.774233,0.652088,8.708271,0.692212,7.313229e+06,414005.020618,9.323136,6.596572e+06,0.645199,...,8.235526e+10,0.660494,-5707.015274,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,2.643
4,2003-5,0.774622,0.652209,8.703393,0.692509,7.322403e+06,414203.609146,9.324709,6.599689e+06,0.645663,...,8.245980e+10,0.648337,-5599.317941,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,2.546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,2022-9,NaN,0.661031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.032
237,2022-10,NaN,0.661074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.573
238,2022-11,NaN,0.661117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.493
239,2022-12,NaN,0.661160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.493


In [8]:
input_data = data.iloc[:split_index + 1,1:-1]
# input_data = data.iloc[:split_index + 1,:]
input_data = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
input_data

,Acre - IDH Longevidade,Acre - IDH Renda,Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH Educacao,Contratação Comercial (Bi R$),...,Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
0,-2.415882,-2.356162,-0.871260,-2.233199,-1.827644,0.550935,-0.265790,-2.404401,-2.033439,1.874594,...,-1.063593,-0.822933,2.782450,0.097384,4.506880,2.067266,2.574314,-2.064648,-2.469876,3.184489
1,-2.372431,-2.318388,-0.874207,-2.196498,-1.807633,0.567407,-0.242610,-2.361200,-2.005361,1.869312,...,-1.063593,-0.818174,2.407943,0.100462,4.328460,1.285816,2.334870,-2.037913,-2.431875,3.029073
2,-2.328980,-2.280615,-0.877154,-2.159796,-1.787622,0.583879,-0.219431,-2.317998,-1.977284,1.866230,...,-1.063593,-0.815398,2.179073,0.072380,4.129086,2.919965,2.221334,-2.011179,-2.393874,2.926505
3,-2.285529,-2.242841,-0.880101,-2.123094,-1.767611,0.600352,-0.196252,-2.274796,-1.949206,1.865900,...,-1.063593,-0.815101,2.077086,0.008167,3.911409,1.142823,2.040542,-1.984445,-2.355872,2.828220
4,-2.242078,-2.205067,-0.883048,-2.086392,-1.747600,0.616824,-0.173072,-2.231595,-1.921128,1.856658,...,-1.063566,-0.812592,1.942128,0.033724,3.663912,2.355956,1.912744,-1.957710,-2.317871,3.036493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,0.848464,0.963521,1.086583,1.093639,0.845955,-0.984945,-1.492171,0.341015,1.238509,-0.624727,...,1.690001,0.667867,-0.689886,2.909383,-0.724085,-1.189161,3.148408,1.378950,0.117681,0.370628
200,0.826218,0.932813,1.079514,1.075237,0.829861,-0.967308,-1.480041,0.326774,1.225428,-0.548742,...,1.717926,0.640959,-0.441954,2.913017,-0.736434,-1.296499,3.219670,1.391539,0.092456,0.427775
201,0.803973,0.902105,1.072445,1.056835,0.813767,-0.949672,-1.467910,0.312533,1.212348,-0.439805,...,1.748758,0.615514,-0.132782,2.818435,-0.738433,-1.444029,3.421082,1.404128,0.067231,0.538287
202,0.781727,0.871397,1.065376,1.038433,0.797673,-0.932035,-1.455780,0.298292,1.199267,-0.327203,...,1.780444,0.593546,0.084061,2.905726,-0.738236,-1.444143,3.373840,1.416717,0.042006,0.666156


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0       2.191
1       2.154
2       2.643
3       2.546
4       3.040
        ...  
236    18.573
237    15.493
238    15.493
239       NaN
240       NaN
Name: Acre - Consumo de Cimento (t), Length: 241, dtype: float64

In [10]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Acre - IDH Longevidade,Acre - IDH Renda,Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH Educacao,Contratação Comercial (Bi R$),...,Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
0,-2.415882,-2.356162,-0.871260,-2.233199,-1.827644,0.550935,-0.265790,-2.404401,-2.033439,1.874594,...,-1.063593,-0.822933,2.782450,0.097384,4.506880,2.067266,2.574314,-2.064648,-2.469876,3.184489
1,-2.372431,-2.318388,-0.874207,-2.196498,-1.807633,0.567407,-0.242610,-2.361200,-2.005361,1.869312,...,-1.063593,-0.818174,2.407943,0.100462,4.328460,1.285816,2.334870,-2.037913,-2.431875,3.029073
2,-2.328980,-2.280615,-0.877154,-2.159796,-1.787622,0.583879,-0.219431,-2.317998,-1.977284,1.866230,...,-1.063593,-0.815398,2.179073,0.072380,4.129086,2.919965,2.221334,-2.011179,-2.393874,2.926505
3,-2.285529,-2.242841,-0.880101,-2.123094,-1.767611,0.600352,-0.196252,-2.274796,-1.949206,1.865900,...,-1.063593,-0.815101,2.077086,0.008167,3.911409,1.142823,2.040542,-1.984445,-2.355872,2.828220
4,-2.242078,-2.205067,-0.883048,-2.086392,-1.747600,0.616824,-0.173072,-2.231595,-1.921128,1.856658,...,-1.063566,-0.812592,1.942128,0.033724,3.663912,2.355956,1.912744,-1.957710,-2.317871,3.036493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,1.369760,1.036081,1.310355,1.290080,1.098918,-1.469526,-1.416170,0.668474,1.230573,-0.685481,...,1.272681,1.521503,-0.742853,0.615006,-0.619453,0.617009,-0.970725,0.980624,0.764972,-1.322806
164,1.363312,1.010393,1.299668,1.287297,1.100953,-1.469505,-1.430567,0.666551,1.227875,-0.757697,...,1.288866,1.512161,-0.787367,0.546355,-0.611176,0.459632,-0.841670,0.991608,0.753473,-1.355523
165,1.356865,0.984705,1.288982,1.284515,1.102988,-1.469485,-1.444964,0.664628,1.225176,-0.844729,...,1.304998,1.500026,-0.757940,0.421319,-0.599627,0.493659,-0.838453,1.002592,0.741973,-1.376175
166,1.350417,0.959017,1.278295,1.281733,1.105023,-1.469464,-1.459361,0.662705,1.222477,-0.914820,...,1.332157,1.486995,-0.917469,0.661467,-0.589237,0.617024,-1.006001,1.013577,0.730473,-1.414146


In [11]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0       2.191000
1       2.154000
2       2.643000
3       2.546000
4       3.040000
         ...    
163    10.885543
164    10.765657
165    10.645771
166    10.525886
167    10.406000
Name: Acre - Consumo de Cimento (t), Length: 168, dtype: float64

In [12]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [13]:
reshaped_train, reshaped_target = create_batches(train_input, train_target, 36, 0)
reshaped_train.shape

(132, 36, 62)

In [14]:
# input para treinamento
test_input = input_data.iloc[train_split-36:split_index + 1]
test_input

,Acre - IDH Longevidade,Acre - IDH Renda,Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - IDH Educacao,Contratação Comercial (Bi R$),...,Títulos – CRI (R$),Títulos – LCI (R$),IPCA - Variação mensal durante o Plano Real (%),NFSP - Fluxo Mensal (Milhões de reais),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%)
132,0.348711,0.619731,0.907236,0.359227,0.953885,-0.502920,0.100422,1.271724,0.448323,1.160295,...,0.467927,0.897908,1.590448,0.225519,-0.298056,-0.311347,-0.077688,0.623805,0.982447,0.041401
133,0.343925,0.600727,0.931034,0.356934,0.960468,-0.563483,0.028258,1.240885,0.450781,1.253961,...,0.505071,0.970274,1.702213,0.413533,-0.281023,-0.313831,-0.030826,0.635788,0.979099,-0.019620
134,0.339139,0.581723,0.954833,0.354641,0.967052,-0.624047,-0.043907,1.210045,0.453240,1.424431,...,0.541297,1.040830,1.677438,0.431751,-0.265304,0.006739,-0.076111,0.647771,0.975750,0.029284
135,0.334353,0.562718,0.978631,0.352348,0.973636,-0.684611,-0.116072,1.179206,0.455698,1.530656,...,0.571731,1.107516,1.638599,0.414613,-0.251648,-0.486922,-0.205002,0.659754,0.972402,0.072516
136,0.329568,0.543714,1.002429,0.350055,0.980219,-0.745175,-0.188236,1.148367,0.458157,1.642943,...,0.611614,1.168977,1.691843,0.566348,-0.231997,-0.559925,-0.132229,0.671737,0.969054,-0.064532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,0.848464,0.963521,1.086583,1.093639,0.845955,-0.984945,-1.492171,0.341015,1.238509,-0.624727,...,1.690001,0.667867,-0.689886,2.909383,-0.724085,-1.189161,3.148408,1.378950,0.117681,0.370628
200,0.826218,0.932813,1.079514,1.075237,0.829861,-0.967308,-1.480041,0.326774,1.225428,-0.548742,...,1.717926,0.640959,-0.441954,2.913017,-0.736434,-1.296499,3.219670,1.391539,0.092456,0.427775
201,0.803973,0.902105,1.072445,1.056835,0.813767,-0.949672,-1.467910,0.312533,1.212348,-0.439805,...,1.748758,0.615514,-0.132782,2.818435,-0.738433,-1.444029,3.421082,1.404128,0.067231,0.538287
202,0.781727,0.871397,1.065376,1.038433,0.797673,-0.932035,-1.455780,0.298292,1.199267,-0.327203,...,1.780444,0.593546,0.084061,2.905726,-0.738236,-1.444143,3.373840,1.416717,0.042006,0.666156


In [15]:
reshaped_test, reshaped_test_target = create_batches(test_input, target_data, 36, 132)
reshaped_test.shape 

(36, 36, 62)

In [16]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * 6 + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [17]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 6)
    target,target_val = validation_splitter(train_target, 6)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [18]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [19]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2569899515, 79503511, 2373691338, 1125792871, 2548416554, 1830902159, 795396835, 995321241, 3025911040, 1092973794, 2836793132, 4173401411, 4246523716, 4197158895, 4143609487, 85276547, 2133998925, 2611121066, 2352834742, 1557904653, 716149784, 4218281449, 1000120339, 155589619, 1227708041, 4195232054, 2009560043, 906907369, 3750089826, 227538425, 2998453831, 632890714, 3222290336, 2921303609, 279296870, 1939435330, 2929853113, 921354232, 143482530, 1381784501, 160756555, 1845769519, 2584848262, 876264324, 2408840230, 2752592923, 4215000474, 4186714319, 829505279, 2542919300, 224634795, 3728703019, 3512425980, 4192276278, 1997858394, 2012711576, 3364523750, 878128128, 1524103726, 2671290502, 3142733890, 274671597, 4252784244, 738081487, 3548525525, 775724453, 4078106562, 1361492864, 1657411855, 3139971408, 3551062542, 4262505162, 685399691, 1396293889, 671557441, 228644643, 4228276485, 648193769, 46325831, 1232695180, 1877854673, 3954783471, 1734999967, 2820906322, 1839531208, 1387224

2023-10-11 19:41:09.229046: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-11 19:41:09.263038: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-11 19:41:09.263307: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

KeyboardInterrupt: 

In [40]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 111ms/step - loss: 144.9001 - val_loss: 49.5176
Epoch 2/10000
4/4 [==============================] - 0s 8ms/step - loss: 31.9040 - val_loss: 9.3846
Epoch 3/10000
4/4 [==============================] - 0s 8ms/step - loss: 11.6235 - val_loss: 8.8948
Epoch 4/10000
4/4 [==============================] - 0s 8ms/step - loss: 11.2149 - val_loss: 7.7510
Epoch 5/10000
4/4 [==============================] - 0s 8ms/step - loss: 10.0479 - val_loss: 7.6480
Epoch 6/10000
4/4 [==============================] - 0s 8ms/step - loss: 10.1339 - val_loss: 9.8269
Epoch 7/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.8513 - val_loss: 5.7827
Epoch 8/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.6235 - val_loss: 5.2278
Epoch 9/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.6145 - val_loss: 5.7401
Epoch 10/10000
4/4 [==============================] - 0s 8ms/step - loss: 9.0088 - val_loss: 5.0647

Epoch 83/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.6496 - val_loss: 5.3868
Epoch 84/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.8996 - val_loss: 5.8691
Epoch 85/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.7610 - val_loss: 5.0552
Epoch 86/10000
4/4 [==============================] - 0s 9ms/step - loss: 7.2368 - val_loss: 6.1969
Epoch 87/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.0725 - val_loss: 6.9058
Epoch 88/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.6659 - val_loss: 5.3902
Epoch 89/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.5848 - val_loss: 5.4667
Epoch 90/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.7657 - val_loss: 5.3567
Epoch 91/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.8553 - val_loss: 5.5584
Epoch 92/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.1303 - val_loss: 4.9718


4/4 [==============================] - 0s 8ms/step - loss: 6.4256 - val_loss: 6.1227
Epoch 246/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.5050 - val_loss: 5.4422
Epoch 247/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.5127 - val_loss: 5.6212
Epoch 248/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.1954 - val_loss: 7.4702
Epoch 249/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.4534 - val_loss: 7.4200
Epoch 250/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.0698 - val_loss: 5.7486
Epoch 251/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.5692 - val_loss: 6.1505
Epoch 252/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.2776 - val_loss: 5.4550
Epoch 253/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.5686 - val_loss: 6.1323
Epoch 254/10000
4/4 [==============================] - 0s 8ms/step - loss: 6.1584 - val_loss: 5.2341
Epoch 

4/4 [==============================] - 0s 8ms/step - loss: 5.7116 - val_loss: 4.2949
Epoch 408/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.8362 - val_loss: 3.7349
Epoch 409/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.9615 - val_loss: 4.8266
Epoch 410/10000
4/4 [==============================] - 0s 7ms/step - loss: 6.0326 - val_loss: 4.7163
Epoch 411/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.9129 - val_loss: 5.3264
Epoch 412/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.4753 - val_loss: 5.1417
Epoch 413/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.1140 - val_loss: 5.6030
Epoch 414/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.7866 - val_loss: 4.7902
Epoch 415/10000
4/4 [==============================] - 0s 7ms/step - loss: 5.3649 - val_loss: 4.3971
Epoch 416/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.5993 - val_loss: 3.8317
Epoch 

4/4 [==============================] - 0s 8ms/step - loss: 5.3712 - val_loss: 4.1992
Epoch 570/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.2006 - val_loss: 6.2364
Epoch 571/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.1343 - val_loss: 4.5228
Epoch 572/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.4549 - val_loss: 5.0518
Epoch 573/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.6642 - val_loss: 4.1934
Epoch 574/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.1190 - val_loss: 5.4921
Epoch 575/10000
4/4 [==============================] - 0s 8ms/step - loss: 4.9647 - val_loss: 3.5743
Epoch 576/10000
4/4 [==============================] - 0s 7ms/step - loss: 5.3270 - val_loss: 4.2720
Epoch 577/10000
4/4 [==============================] - 0s 8ms/step - loss: 5.2545 - val_loss: 4.4976
Epoch 578/10000
4/4 [==============================] - 0s 7ms/step - loss: 4.9270 - val_loss: 3.8716
Epoch 

...

4/4 [==============================] - 0s 8ms/step - loss: 2.1869 - val_loss: 4.0064
Epoch 894/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.9067 - val_loss: 3.1329
Epoch 895/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.1930 - val_loss: 3.7356
Epoch 896/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.1829 - val_loss: 3.7954
Epoch 897/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.0757 - val_loss: 3.0279
Epoch 898/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.0836 - val_loss: 3.2213
Epoch 899/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.9685 - val_loss: 2.5340
Epoch 900/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.4066 - val_loss: 2.9686
Epoch 901/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.8722 - val_loss: 2.4896
Epoch 902/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.3704 - val_loss: 5.4372
Epoch 

4/4 [==============================] - 0s 8ms/step - loss: 2.4283 - val_loss: 4.5639
Epoch 1055/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.7979 - val_loss: 2.8442
Epoch 1056/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.6768 - val_loss: 2.5386
Epoch 1057/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.7813 - val_loss: 3.3003
Epoch 1058/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.8015 - val_loss: 4.4171
Epoch 1059/10000
4/4 [==============================] - 0s 8ms/step - loss: 2.0225 - val_loss: 3.8192
Epoch 1060/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.8298 - val_loss: 3.5797
Epoch 1061/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.7849 - val_loss: 3.8798
Epoch 1062/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.7096 - val_loss: 3.7294
Epoch 1063/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.5472 - val_loss: 4.41

4/4 [==============================] - 0s 7ms/step - loss: 1.5047 - val_loss: 5.3118
Epoch 1215/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.4124 - val_loss: 3.2788
Epoch 1216/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.5531 - val_loss: 2.9412
Epoch 1217/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.4318 - val_loss: 3.2029
Epoch 1218/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.2597 - val_loss: 3.4525
Epoch 1219/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.4806 - val_loss: 3.9187
Epoch 1220/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.5323 - val_loss: 3.5522
Epoch 1221/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.6843 - val_loss: 4.2789
Epoch 1222/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.5015 - val_loss: 2.1243
Epoch 1223/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.6801 - val_loss: 3.67

4/4 [==============================] - 0s 8ms/step - loss: 1.1228 - val_loss: 3.4297
Epoch 1375/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.3423 - val_loss: 2.8122
Epoch 1376/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.2985 - val_loss: 2.9045
Epoch 1377/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.3006 - val_loss: 3.4166
Epoch 1378/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.2583 - val_loss: 3.9509
Epoch 1379/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.2264 - val_loss: 3.3736
Epoch 1380/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.2548 - val_loss: 3.6205
Epoch 1381/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.1502 - val_loss: 3.8210
Epoch 1382/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.1442 - val_loss: 3.9033
Epoch 1383/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.4017 - val_loss: 3.85

4/4 [==============================] - 0s 8ms/step - loss: 0.9285 - val_loss: 3.5713
Epoch 1535/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.0961 - val_loss: 5.1281
Epoch 1536/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.0894 - val_loss: 3.5660
Epoch 1537/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.4782 - val_loss: 3.9824
Epoch 1538/10000
4/4 [==============================] - 0s 7ms/step - loss: 1.4724 - val_loss: 2.6568
Epoch 1539/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.3839 - val_loss: 5.6615
Epoch 1540/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.2656 - val_loss: 2.5849
Epoch 1541/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.1383 - val_loss: 4.8450
Epoch 1542/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.2464 - val_loss: 6.1729
Epoch 1543/10000
4/4 [==============================] - 0s 8ms/step - loss: 1.0422 - val_loss: 2.80

In [41]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    for i in range(len(test_target)):
        input_test = test_input[i:i+1]
        prediction = model.predict(input_test)
        error = np.abs(prediction - test_target[start_target + i])
        errors.append(error)
        error_percent.append(error / test_target[start_target + i])
        print(f"Month-{i + 1} - Error: {error}")
    
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [42]:
errors, mae, mape = mae_mape_calculator(trained_model, 
                                        reshaped_test, 
                                        reshaped_test_target, 0)

1/1 [==============================] - 0s 285ms/step
Month-1 - Error: [[1.6619129]]
1/1 [==============================] - 0s 14ms/step
Month-2 - Error: [[1.1468801]]
1/1 [==============================] - 0s 14ms/step
Month-3 - Error: [[0.51917267]]
1/1 [==============================] - 0s 14ms/step
Month-4 - Error: [[3.8361216]]
1/1 [==============================] - 0s 14ms/step
Month-5 - Error: [[2.6262684]]
1/1 [==============================] - 0s 14ms/step
Month-6 - Error: [[2.9722834]]
1/1 [==============================] - 0s 14ms/step
Month-7 - Error: [[5.2613926]]
1/1 [==============================] - 0s 14ms/step
Month-8 - Error: [[4.300516]]
1/1 [==============================] - 0s 14ms/step
Month-9 - Error: [[4.1583157]]
1/1 [==============================] - 0s 14ms/step
Month-10 - Error: [[2.174861]]
1/1 [==============================] - 0s 14ms/step
Month-11 - Error: [[1.3369923]]
1/1 [==============================] - 0s 14ms/step
Month-12 - Error: [[2.5113592]]
1

In [43]:
display(mae)
display(mape)

4.6796603

0.3468182

In [44]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}")
            target_sum = 0
            prediction_sum = 0
            
        input_test = test_input[i:i+1]
        prediction = model.predict(input_test)
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [45]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 0)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[127.34885]] - Target[153.199]| =  Error: [[25.850159]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[129.76167]] - Target[146.76999999999998]| =  Error: [[17.008331]]
1/1 [==============================] - 0s 14ms/step
Ano-11: |Prediction[[54.703346]] - Target[155.80599999999995]| =  Error: [[101.10265]]


[array([[25.850159]], dtype=float32),
 array([[17.008331]], dtype=float32),
 array([[101.10265]], dtype=float32)]

47.98705

0.31117365